In [2]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

    
# Installing the Reinforcement Learning library directly from github.
# %pip install git+https://github.com/lvwerra/trl.git@25fa1bd  

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.9 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
    
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [2]:
import os

# create a folder for the script files
script_folder = '../src'
output_folder = '../output'
os.makedirs(script_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
print(script_folder, 'folder created')

../src folder created


In [ ]:
%%writefile ../src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
  - defaults
dependencies:
  - pip
  - pip:
    - torch==1.13.1
    - rouge_score==0.1.2
  - python=3.10.11
  - datasets==2.17.0
  - torchdata==0.5.1
  - transformers==4.27.2
  - evaluate==0.4.0
  - loralib==0.1.1
  - peft==0.3.0
  - mlflow

In [ ]:
# create new environment using base Docker image and conda specs.
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:latest",
    conda_file="../src/conda-env.yml",
    name="docker-image-llm",
    description="llm from docker",
)
ml_client.environments.create_or_update(env_docker_conda)


In [16]:
%%writefile $script_folder/sentiment_prediction.py
import mlflow
import argparse
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def predict_sentiment(dialogue, output_path, model_name='google/flan-t5-base'):
    # Initialize tokeniazer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Constructing a 5-shot prompt with examples
    start_prompt = '''Provide Sentiment for the following comment/conversation (possible sentiments: Positive, Negative, Neutral):

    Comment: "I love sunny days, they make me feel so happy!"
    Sentiment: Positive

    Comment: "This is the worst experience of my life, I'm so disappointed."
    Sentiment: Negative

    Comment: "I'm not sure how I feel about this new policy. It might be good or bad."
    Sentiment: Neutral

    Comment: "The service at this restaurant was fantastic, best dinner ever!"
    Sentiment: Positive

    Comment: "I waited for an hour and my order was still wrong."
    Sentiment: Negative

    Comment: '''
    
    end_prompt = '\nSentiment: '
    
    # Construct the full prompt with the user-provided dialogue
    prompt = start_prompt + '"' + dialogue + '"' + end_prompt 

    # Tokenize input dialogue
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate prediction
    output = model.generate(inputs['input_ids'], max_new_tokens=50)
    
    # Decode and print the prediction
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Write the predicted sentiment to the specified output file
    with open(output_path, 'w') as f:
        f.write('Text: ' + dialogue + '\nPredicted Sentiment: ' + decoded_output + '\n')

def main():
    # enable autologging
    mlflow.autolog()
    
    parser = argparse.ArgumentParser(description="Predict sentiment from input dialogue")
    parser.add_argument("--dialogue", type=str, required=True, help="Input dialogue for sentiment prediction")
    parser.add_argument("--output", type=str, required=True, help="Output file path for sentiment prediction")
    
    args = parser.parse_args()

    # Predict sentiment and write to output
    predict_sentiment(args.dialogue, args.output)

if __name__ == "__main__":
    main()


Overwriting ../src/sentiment_prediction.py


In [4]:
#Test script
#!python $script_folder/sentiment_prediction.py --dialogue "I love this book!" --output $output_folder"/output.txt"


2024/02/24 06:28:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2024-02-24 06:28:48.480926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-24 06:28:52.141555: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-24 06:28:53.196121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loade

In [ ]:
from azure.ai.ml.entities import ComputeInstance


ci = ComputeInstance(
    name="compute-instance", 
    size="Standard_E4ds_v4"
)
ml_client.begin_create_or_update(ci).result()


In [22]:
%%writefile ../sentiment_prediction.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: sentiment_prediction_merged
display_name: Sentiment Prediction with Integrated Tokenization
version: 3
type: command
inputs:
  dialogue: 
    type: string
outputs:
  sentiment_output:
    type: uri_file
code: ./src
environment: azureml:docker-image-llm@latest
compute: azureml:cpu-cluster
command: >-
  python sentiment_prediction.py 
  --dialogue ${{inputs.dialogue}}
  --output ${{outputs.sentiment_output}}


Overwriting ../sentiment_prediction.yml


In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def sentiment_prediction(pipeline_job_input):
    sentiment = predict_sentiment_segment(dialogue=pipeline_job_input)

    return {
        "pipeline_job_predict_sentiment_data": sentiment.outputs.sentiment_output,
        
    }

# Example usage with a direct string input for the dialogue
pipeline_job = sentiment_prediction(pipeline_job_input='"Movie is rathet bad!"')
pipeline_job.settings.default_compute = "compute-instance"

In [19]:
from azure.ai.ml import load_component
parent_dir = ""

predict_sentiment_segment = load_component(source=parent_dir + "../sentiment_prediction.yml")

# register component
prep = ml_client.components.create_or_update(predict_sentiment_segment, version='6')

In [20]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def sentiment_prediction(pipeline_job_input):
    sentiment = predict_sentiment_segment(dialogue=pipeline_job_input)

    return {
        "pipeline_job_predict_sentiment_data": sentiment.outputs.sentiment_output,
        
    }

# Example usage with a direct string input for the dialogue
pipeline_job = sentiment_prediction(pipeline_job_input='"Movie is rathet bad!"')
pipeline_job.settings.default_compute = "compute-instance"

In [21]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="sentiment_prediction"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
sentiment_prediction,busy_chicken_qw3fty2g06,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# #CLI2 version of creating component and pipeline
# !az extension add --name ml -y
# output = %sx az ml component list \
#         --resource-group "cloud-shell-storage-southeastasia" \
#         --workspace-name "oksana_ml"
# print(output)
# !az ml component create --file ../sentiment_prediction.yml
# !az ml job create --file ../pipeline_sentiment_prediction.yml

In [25]:
# Script to register the 'google/flan-t5-base' model for sentiment analysis with MLflow and Azure ML for future online inference

# Importing necessary libraries for Azure ML, MLflow, and model loading
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential
import mlflow
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

def register_model_with_mlflow(model_name='google/flan-t5-base', model_save_path='saved_models/flan_t5_base'):
    """
    Load a pre-trained model and tokenizer from Hugging Face, save them locally,
    and register the model with MLflow for tracking and version control.
    
    Parameters:
    - model_name (str): Identifier for the pre-trained model on Hugging Face.
    - model_save_path (str): Local directory path for saving the model and tokenizer.
    
    Returns:
    - mlflow_model_name (str): The name of the model registered in MLflow, adjusted for compatibility.
    """
    # Adjusting model_name for MLflow compatibility (replacing '/' with '_')
    mlflow_model_name = model_name.replace('/', '_')

    # Loading the tokenizer and model from Hugging Face Transformers
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    # Saving the tokenizer and model locally
    tokenizer.save_pretrained(model_save_path)
    model.save_pretrained(model_save_path)
    
    # Logging the model to MLflow with the adjusted name
    mlflow.pytorch.log_model(pytorch_model=model, artifact_path="models", registered_model_name=mlflow_model_name)
    
    return mlflow_model_name

def main():
    # Setting up MLflow: Configuring the tracking URI and experiment name
    mlflow.set_tracking_uri("your_mlflow_tracking_uri")  # Replace with your actual MLflow tracking URI
    mlflow.set_experiment("ModelRegistrationExperiment")

    # Starting an MLflow run to log model information
    with mlflow.start_run():
        mlflow_model_name = register_model_with_mlflow()

    # Fetching the latest run from the experiment for artifact URI
    experiment = mlflow.get_experiment_by_name("ModelRegistrationExperiment")
    runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
    latest_run_id = runs.iloc[0]['run_id']  # Assuming the latest run contains the relevant model

    # Extracting the model artifact URI from the latest run information
    run_info = mlflow.get_run(latest_run_id)
    model_artifact_uri = run_info.info.artifact_uri + "/models"


    # Defining Azure ML model metadata for registration
    azure_model = Model(
        name=mlflow_model_name,  # Name of the model in Azure ML registry
        description="Pre-trained sentiment analysis model registered via MLflow.",
        # type="mlflow_model",  # Model type indicating it's tracked by MLflow
        type=AssetTypes.MLFLOW_MODEL,
        path=model_artifact_uri,  # Path to the model artifact for registration
        version="4"  # Model version
    )
    
    # Registering the model in Azure ML
    registered_model = ml_client.models.create_or_update(azure_model)
    print(f"Model registered successfully in Azure ML: {registered_model.name}, Version: {registered_model.version}")

if __name__ == "__main__":
    main()


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'google_flan-t5-base' already exists. Creating a new version of this model...
2024/02/28 08:46:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: google_flan-t5-base, version 12
Created version '12'

Model registered successfully in Azure ML: google_flan-t5-base, Version: 4


In [3]:
#Define and create an endpoint
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for sentiment extraction",
    auth_mode="key",
)

In [4]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02290657909631.australiasoutheast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02290657909631', 'description': 'Online endpoint for sentiment extraction', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/resourcegroups/cloud-shell-storage-southeastasia/providers/microsoft.machinelearningservices/workspaces/oksana_ml/onlineendpoints/endpoint-02290657909631', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/71dd2dd9-4027-4b07-a6aa-e98b8b31e8cc/providers/Microsoft.MachineLearningServices/locations/australiasoutheast/mfeOperationsStatus/oe:ce49baec-5e26-4e4d-9649-a752220012af:b0f9cdd3-a918-48d8-898d-ffe73bdb7af2?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/sub

In [ ]:
#Configure the deployment
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path="azureml://locations/australiasoutheast/workspaces/ce49baec-5e26-4e4d-9649-a752220012af/models/google_flan-t5-base/versions/4",
    type=AssetTypes.MLFLOW_MODEL,
    description="sentiment extraction mlflow model",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [ ]:
#Create deployment
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()